## **LAB 08 : ENSEMBLE LEARNING**



In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB, MultinomialNB,CategoricalNB
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score   

### **Implementation of stacking , with max voting as classification model**

In [10]:
def max_voting(output):
  votes={}
  for elem in output:
    if elem in votes:
      votes[elem]+=1
    else :
      votes[elem]=1
  final_output=-1
  max_votes=-1
  for elem in votes:
    if max_votes<votes[elem]:
      max_votes=votes[elem]
      final_output=elem
      pass
    pass
  print(votes)
  return final_output,max_votes
  pass


def get_data_set():
  weather = ['Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy','Rainy', 'Overcast','Sunny', 'Sunny', 'Rainy', 'Sunny', 'Overcast', 'Overcast', 'Rainy']
  temp = ['Hot','Hot','Hot','Mild','Cool','Cool','Cool','Mild','Cool','Mild','Mild','Mild','Hot','Mild']
  play=['No','No','Yes','Yes','Yes','No','Yes','No','Yes','Yes','Yes','Yes','Yes','No']
  le=preprocessing.LabelEncoder()
  weather_encoded=le.fit_transform(weather)
  temp_encoded=le.fit_transform(temp)
  play_encoded=le.fit_transform(play)
  features=tuple(zip(weather_encoded,temp_encoded))
  return features,play_encoded
  pass


def get_individual_classifiers(x_train,y_train,no_of_models=10):
  individual_classifiers=[]
  for _ in range(no_of_models):
    
    new_model=CategoricalNB(alpha=1)
    new_model.fit(x_train,y_train)
    individual_classifiers.append(new_model)
    pass
  return individual_classifiers
  pass


def individual_predict(individual_classifiers,input):
  output=[]
  for classifier in individual_classifiers:
    curr_output=list(classifier.predict(input))
    output=output+curr_output
    pass
  return output
  pass


def get_meta_classifier(x,y):
  individual_classifiers=get_individual_classifiers(x_train,y_train)
  output=individual_predict(individual_classifiers,x)
  meta_x=output
  meta_y=[]
  div=len(meta_x)//len(x)
  for _ in range(div):
    meta_y=meta_y+list(y)
  meta_classifier=LogisticRegression(random_state=0).fit(x, y)
  return meta_classifier
  pass


def get_accuracy(y_pred,y_test):
  
  return accuracy_score(y_pred,y_test)
  pass

x,y=get_data_set() 
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=137,test_size=0.50) 
meta_classifier=get_meta_classifier(x_train,y_train)
y_pred=meta_classifier.predict(x_test)
print(y_pred)
print("accuracy : ",get_accuracy(y_pred,y_test))


[1 1 1 1 1 0 0]
accuracy :  0.7142857142857143


### **Stacking using regression and low diabetes database**

In [7]:
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB, MultinomialNB,CategoricalNB
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score   
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


# def max_voting(output):
#   votes={}
#   for elem in output:
#     if elem in votes:
#       votes[elem]+=1
#     else :
#       votes[elem]=1
  
#   final_output=-1
#   max_votes=-1
#   for elem in votes:
#     if max_votes<votes[elem]:
#       max_votes=votes[elem]
#       final_output=elem
#       pass
#     pass
#   print(votes)
#   return final_output,max_votes
#   pass


def get_data_set():
  import pandas as pd
  dataset=load_diabetes()
  x=dataset.data
  y=dataset.target
  return x,y
  pass


def get_individual_classifiers(x_train,y_train,no_of_models=10):
  individual_classifiers=[]
  for _ in range(no_of_models):
    new_model=LinearRegression()
    new_model.fit(x_train,y_train)
    individual_classifiers.append(new_model)
    pass
  return individual_classifiers
  pass


def individual_predict(individual_classifiers,input):
  output=[]
  for classifier in individual_classifiers:
    curr_output=list(classifier.predict(input))
    output=output+curr_output
    pass
  return output
  pass


def get_meta_classifier(x,y):
  individual_classifiers=get_individual_classifiers(x_train,y_train)
  output=individual_predict(individual_classifiers,x)
  meta_x=output
  meta_y=[]
  div=len(meta_x)//len(x)
  for _ in range(div):
    meta_y=meta_y+list(y)
  meta_classifier=LinearRegression().fit(x, y)
  return meta_classifier
  pass


def get_accuracy(y_pred,y_test):
  return accuracy_score(y_pred,y_test)
  pass


def get_mse(y_pred,y_test):
  return mean_squared_error(y_pred,y_test)
  pass


x,y=get_data_set()
# print(x,y)
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=137,test_size=0.50)
meta_classifier=get_meta_classifier(x_train,y_train)
y_pred=meta_classifier.predict(x_test)
print("mse loss : ",get_mse(y_pred,y_test))
# x_1,y_1=individual_predict(individual_classifiers,[[1,2]])
# print(x_1,y_1)


mse loss :  2679.3209640213354


### **Adaboost classifier**

In [11]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score   
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix


def get_data_set():
  import pandas as pd
  dataset=load_breast_cancer()
  x=dataset.data
  y=dataset.target
  return x,y
  pass


def get_classifier(x,y):
  return AdaBoostClassifier().fit(x,y)


def get_accuracy(y_pred,y_test):
  return accuracy_score(y_pred,y_test)
  pass


def print_classification_report(y_pred,y_test):
  print("confusion matrix of addboostclassification")
  cmat=confusion_matrix(y_test,y_pred)
  print(cmat)
  pass
x,y=get_data_set()
# print(x,y)
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=137,test_size=0.50)
classifier=get_classifier(x_train,y_train)
y_pred=classifier.predict(x_test)
print("accuracy : ",get_accuracy(y_pred,y_test))
print_classification_report(y_test,y_pred)


accuracy :  0.9789473684210527
confusion matrix of addboostclassification
[[103   2]
 [  4 176]]


### **Use StackingClassifier from sklearn to implement the same on cancer dataset**

In [13]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score   
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier


def get_data_set():
  import pandas as pd
  dataset=load_breast_cancer()
  x=dataset.data
  y=dataset.target
  return x,y
  pass


def get_classifier(x,y):
  return BaggingClassifier().fit(x,y)


def get_accuracy(y_pred,y_test):
  return accuracy_score(y_pred,y_test)
  pass


def print_classification_report(y_pred,y_test):
  print("confusion matrix of bagging on descision tree classifier : ")
  cmat=confusion_matrix(y_test,y_pred)
  print(cmat)
  pass


x,y=get_data_set()
# print(x,y)
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=137,test_size=0.50)
classifier=get_classifier(x_train,y_train)
y_pred=classifier.predict(x_test)
print("accuracy : ",get_accuracy(y_pred,y_test))
print_classification_report(y_test,y_pred)


accuracy :  0.9543859649122807
confusion matrix of bagging on descision tree classifier : 
[[100   6]
 [  7 172]]
